Lectura de los datos 

In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

def extraer_imagenes(directorio=r'/home/cursos/ima543_2025_1/ima543_share/Datasets/FER/test'):
    """
    Carga todas las imágenes desde el directorio especificado, normaliza las imágenes y devuelve las matrices listas para entrenamiento o prueba.
    """
    clases = []
    imagenes = []
    lista_carpetas = os.listdir(directorio)
    
    k = 0
    for carpeta_clase in lista_carpetas:
        ruta_carpeta = os.path.join(directorio, carpeta_clase)
        if not os.path.isdir(ruta_carpeta):
            continue  # Ignorar si no es carpeta
        fotos = os.listdir(ruta_carpeta)
        for foto in fotos:
            img_path = os.path.join(ruta_carpeta, foto)
            imagen = image.load_img(img_path, target_size=(256, 256))
            imagenes.append(np.array(imagen))
            clases.append(k)
        k += 1

    # Conversión a numpy
    X = np.array(imagenes, dtype=np.float32)
    y = np.array(clases)

    # Formato
    image_size = X.shape[1]
    X = np.reshape(X, [-1, image_size, image_size, 3])

    # Normalizar
    X = X / 255.0

    # One-hot encoding
    y = to_categorical(y)

    return X, y


In [7]:
# Cargar datos de entrenamiento
X_train, y_train = extraer_imagenes(
    directorio=r'/home/cursos/ima543_2025_1/ima543_share/Datasets/FER/train'
)

# Cargar datos de prueba
X_test, y_test = extraer_imagenes(
    directorio=r'/home/cursos/ima543_2025_1/ima543_share/Datasets/FER/test'
)


KeyboardInterrupt



Procesando data en DenseNet

In [ ]:
"""Trains a 100-Layer DenseNet on tus datos personalizados."""
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

import os
import numpy as np
import math

# Importa aquí tu función (defínela antes o en un módulo aparte)
from extraer_imagenes import extraer_imagenes

# --- Parámetros de entrenamiento ---
batch_size = 32
#epochs = 200
epochs = 1
data_augmentation = True

# --- Parámetros de la red ---
# Growth rate | Depth   |  Accuracy (paper) ...
growth_rate = 12
depth = 100
num_dense_blocks = 4
use_max_pool = False

num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)
num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

# --- Carga tus datos desde carpetas Train / Test ---
train_dir = r'/home/cursos/ima543_2025_1/ima543_share/Datasets/FER/train'
test_dir  = r'/home/cursos/ima543_2025_1/ima543_share/Datasets/FER/test'

x_train, y_train = extraer_imagenes(directorio=train_dir)
x_test,  y_test  = extraer_imagenes(directorio=test_dir)

# Número de clases y forma de entrada
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]  # e.g. (256, 256, 3)

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# --- Definición de la función de LR schedule (queda igual) ---
def lr_schedule(epoch):
    """Learning Rate Schedule"""
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print(f'Epoch {epoch} — Learning rate: {lr}')
    return lr

# (Aquí seguiría la construcción de tu modelo DenseNet, 
# los callbacks, el compilado y el fit con ImageDataGenerator si lo usas.)

# Ejemplo de fit con data augmentation:
if data_augmentation:
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
    )
    datagen.fit(x_train)
    model.fit(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        validation_data=(x_test, y_test),
        epochs=epochs,
        callbacks=[
            LearningRateScheduler(lr_schedule),
            ModelCheckpoint(...),
            ReduceLROnPlateau(...)
        ]
    )
else:
    model.fit(
        x_train, y_train,
        batch_size=batch_size,
        validation_data=(x_test, y_test),
        epochs=epochs,
        callbacks=[LearningRateScheduler(lr_schedule)]
    )


In [ ]:
num_bottleneck_layers

In [ ]:
# start model definition
# densenet CNNs (composite function) are made of BN-ReLU-Conv2D
inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,kernel_size=3,padding='same',kernel_initializer='he_normal')(x)
x = concatenate([inputs, x])

# stack of dense blocks bridged by transition layers
for i in range(num_dense_blocks):
    # a dense block is a stack of bottleneck layers
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,kernel_size=1,padding='same',kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,kernel_size=3,padding='same',kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        x = concatenate([x, y])

    # no transition layer after the last dense block
    if i == num_dense_blocks - 1:
        continue

    # transition layer compresses num of feature maps and reduces the size by 2
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,kernel_size=1,padding='same',kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)


# add classifier on top
# after average pooling, size of feature map is 1 x 1
x = AveragePooling2D(pool_size=4)(x)
y = Flatten()(x)
outputs = Dense(num_classes,kernel_initializer='he_normal',activation='softmax')(y)

# instantiate and compile model
# orig paper uses SGD but RMSprop works better for DenseNet
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(1e-3),metrics=['acc'])
model.summary()

In [ ]:
# prepare model model saving directory
save_dir = os.path.join(os.getcwd(), 'saved_models_'+ str(epochs) +'_epocas')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate reducer
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc',verbose=2,save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

import time

start=time.time()

# run training, with or without data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,
              validation_data=(x_test, y_test),shuffle=True,callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # preprocessing  and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(x_train)

    steps_per_epoch = math.ceil(len(x_train) / batch_size)
    # fit the model on the batches generated by datagen.flow().
    model.fit(x=datagen.flow(x_train, y_train, batch_size=batch_size),verbose=2,epochs=epochs,
              validation_data=(x_test, y_test),steps_per_epoch=steps_per_epoch,callbacks=callbacks)

fin=time.time()
print('(RUNNING TIME: ' + str(fin-start) )
# score trained model
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de pérdida (loss)
plt.figure(figsize=(8, 6))
plt.plot(history.history["loss"], label="Pérdida Entrenamiento (2 entradas)")
plt.plot(history.history["val_loss"], label="Pérdida Validación (2 entradas)")
plt.title("Pérdida durante el entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.legend(loc="best")
plt.grid()
plt.savefig("pérdida_RNN.jpg", bbox_inches="tight", dpi=300)
plt.show()

# Gráfico de precisión (accuracy)
plt.figure(figsize=(8, 6))
plt.plot(history.history["accuracy"], label="Precisión Entrenamiento (2 entradas)")
plt.plot(history.history["val_accuracy"], label="Precisión Validación (2 entradas)")
plt.title("Precisión durante el entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.legend(loc="best")
plt.grid()
plt.savefig("precisión_RNN.jpg", bbox_inches="tight", dpi=300)
plt.show()